## Load dataset

In [1]:
import glob
import pandas as pd

files = glob.glob('../evaluate_results/outer/*.csv', recursive=True)

print(len(files), 'files')
files[0:3]

94 files


["../evaluate_results/outer/KNeighborsClassifier-split_x_y_with_projection-{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60, 80, 100], 'algorithm': ['brute'], 'metric': ['hamming']}-(3 of 5).csv",
 "../evaluate_results/outer/SVC-split_x_y_normalized_function-{'C': [1e-05, 0.001, 0.1, 10.0, 1000.0, 100000.0], 'gamma': [1e-05, 0.001, 0.1, 10.0, 1000.0, 100000.0, 'scale'], 'kernel': ['rbf'], 'probability': [True]}-(3 of 5).csv",
 '../evaluate_results/outer/LogisticRegression-split_x_y_split_with_one_hot_encoding-{}-(2 of 5).csv']

In [2]:
# Read all data
data = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data.head(2)

,best_params,column,i_outer,metric,model,params,split_method,value
0,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",0,2,accuracy,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",split_x_y_with_projection,0.296296
1,"{'algorithm': 'brute', 'metric': 'hamming', 'n...",1,2,accuracy,KNeighborsClassifier,"{'n_neighbors': [1, 5, 10, 15, 20, 25, 40, 60,...",split_x_y_with_projection,0.099537


## Results

In [3]:
result = data.groupby(['model', 'split_method', 'metric'])['value'].agg(["mean", "std"])
result.head(3)

mean       std
model                split_method metric                      
KNeighborsClassifier split_x_y    accuracy  0.356083  0.131148
                                  hit@5     0.538557  0.122381
                                  map@5     0.318705  0.153412

## Result as pivot table

In [4]:
result.pivot_table(index=['model', 'split_method'], columns='metric', values=['mean', 'std'])

mean  \
metric                                                                   accuracy   
model                split_method                                                   
KNeighborsClassifier split_x_y                                           0.356083   
                     split_x_y_normalized_function                       0.356083   
                     split_x_y_split_with_one_hot_encoding               0.356083   
                     split_x_y_split_with_one_hot_encoding_and_proje...  0.249185   
                     split_x_y_with_projection                           0.249185   
                     split_x_y_with_random_matrix                        0.249185   
                     split_x_y_word2vec                                  0.236234   
LogisticRegression   split_x_y                                           0.305872   
                     split_x_y_normalized_function                       0.302246   
                     split_x_y_split_with_one_hot_encoding               0.389859   
                     split_x_y_split_with_one_hot_encoding_and_proje...  0.149316   
                     split_x_y_with_projection                           0.002160   
                     split_x_y_with_random_matrix                        0.305640   
                     split_x_y_word2vec                                  0.089689   
SVC                  split_x_y                                           0.330628   
                     split_x_y_normalized_function                       0.329782   
                     split_x_y_split_with_one_hot_encoding               0.396464   
                     split_x_y_with_projection                           0.041022   
                     split_x_y_with_random_matrix                        0.320836   

                                                                                   \
metric                                                                      hit@5   
model                split_method                                                   
KNeighborsClassifier split_x_y                                           0.538557   
                     split_x_y_normalized_function                       0.538557   
                     split_x_y_split_with_one_hot_encoding               0.538557   
                     split_x_y_split_with_one_hot_encoding_and_proje...  0.430046   
                     split_x_y_with_projection                           0.430046   
                     split_x_y_with_random_matrix                        0.430200   
                     split_x_y_word2vec                                  0.417163   
LogisticRegression   split_x_y                                           0.499147   
                     split_x_y_normalized_function                       0.497915   
                     split_x_y_split_with_one_hot_encoding               0.593544   
                     split_x_y_split_with_one_hot_encoding_and_proje...  0.305330   
                     split_x_y_with_projection                           0.110905   
                     split_x_y_with_random_matrix                        0.499688   
                     split_x_y_word2vec                                  0.176459   
SVC                  split_x_y                                           0.539633   
                     split_x_y_normalized_function                       0.542797   
                     split_x_y_split_with_one_hot_encoding               0.583859   
                     split_x_y_with_projection                           0.379374   
                     split_x_y_with_random_matrix                        0.514803   

                                                                                   \
metric                                                                      map@5   
model                split_method                                                   
KNeighborsClassifier split_x_y                                           0.31870

### Params selected

In [5]:
pd.set_option('max_colwidth', 100)
data.sort_values(['model', 'metric', 'split_method'])[['model', 'metric', 'best_params']].drop_duplicates()

,model,metric,best_params
0,KNeighborsClassifier,accuracy,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 15}"
0,KNeighborsClassifier,accuracy,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 25}"
0,KNeighborsClassifier,accuracy,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 100}"
0,KNeighborsClassifier,accuracy,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 60}"
6,KNeighborsClassifier,hit@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 60}"
6,KNeighborsClassifier,hit@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 25}"
6,KNeighborsClassifier,hit@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 40}"
6,KNeighborsClassifier,hit@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 100}"
6,KNeighborsClassifier,hit@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 80}"
24,KNeighborsClassifier,map@5,"{'algorithm': 'brute', 'metric': 'hamming', 'n_neighbors': 80}"
